In [16]:
import sqlite3
import pandas as pd
csv = r"Test.csv"

In [25]:
def green(x):
    if x == 'Yes' or x == 'No': #Converts Yes to 1 and No to 0
        return int(x == 'Yes')
    if x == 'Male' or x == 'Female': #Converts Male to 1 and No to 0
        return int(x == 'Male')
    if 'No' in str(x): #Anything including No to -1, not eligible in the first place, consider leaving as 0 instead
        return -1
    else: #Leaving currently non quantifiable values as strings
        return x
df = pd.read_csv(csv)
for col in df.columns:
    if col != 'InternetService':
        df[col] = df[col].apply(green) #Apply all the conversions by iterating through columns
        
conn = sqlite3.connect('mongey.db') #Convert new data into a db, doesn't overwrite old CSV
df.to_sql('CustomerInfo', conn, if_exists='replace', index=False)
conn.close()


7043